# Gmail + Groq AI Chat Assistant

Paste your code here or use the provided structure.

In [ ]:
pip install ipywidgets pandas google-auth google-auth-oauthlib google-auth-httplib2 google-apipython-client groq

import os

import pickle

from google.auth.transport.requests import Request

from google.oauth2.credentials import Credentials

from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def authenticate_gmail():

creds = None

Load existing token if available

if os.path.exists("token.pkl"):

with open("token.pkl", "rb") as token:

creds = pickle.load(token)

If no valid credentials, request new authentication

if not creds or not creds.valid:

if creds and creds.expired and creds.refresh_token:

creds.refresh(Request())

else:

flow = InstalledAppFlow.from_client_secrets_file(

"C:/Users/gopis/OneDrive/Desktop/New folder/client_secret_597047056700-

0718t8hnr4eq5llb1rs5nn3h9ftboos8.apps.googleusercontent.com.json", # Make sure this file exists

in your directory

SCOPES

)

creds = flow.run_local_server(port=0)

Save token for future use

with open("token.pkl", "wb") as token:

pickle.dump(creds, token)

return creds

Run the function to authenticate

authenticate_gmail()
def fetch_emails(num_emails=20):

"""Fetches a list of emails and stores their subject and content."""

creds = authenticate_gmail()

service = build("gmail", "v1", credentials=creds)

results = service.users().messages().list(userId="me", maxResults=num_emails).execute()

messages = results.get("messages", [])

email_data = []

for msg in messages:

msg_data = service.users().messages().get(userId="me", id=msg["id"]).execute()

subject = "No Subject"

email_content = extract_body(msg_data["payload"])

Extract subject

for part in msg_data["payload"]["headers"]:

if part["name"] == "Subject":

subject = part["value"]

email_data.append({"subject": subject, "content": email_content})

return email_data

!pip install ipywidgets

import base64

import re

from googleapiclient.discovery import build

from groq import Groq

import ipywidgets as widgets

from IPython.display import display

? Groq API Key (Already provided)

GROQ_API_KEY =os.getenv("GROQ_API_KEY")

client = Groq(api_key=GROQ_API_KEY)

? UI Components (Chat Input & Output)

chat_history = widgets.Output()

user_input = widgets.Text(placeholder="Ask AI about emails...")

send_button = widgets.Button(description="Send")

def extract_body(payload):

"""Extracts the plain text body from a Gmail API email payload."""

body = ""

if "parts" in payload:
for part in payload["parts"]:

if part["mimeType"] == "text/plain":

body_data = part["body"].get("data", "")

body = base64.urlsafe_b64decode(body_data).decode("utf-8")

break # Stop after finding the first plain text body

elif payload["mimeType"] == "text/plain":

body_data = payload["body"].get("data", "")

body = base64.urlsafe_b64decode(body_data).decode("utf-8")

return body.strip() if body else "No content found"

? Function to Clean and Extract Email Content

def clean_email_content(text):

"""Removes unwanted links and formatting from email content."""

text = re.sub(r'http[s]?://\S+', '', text) # Remove links

text = re.sub(r'<\S+>', '', text) # Remove <links>

text = re.sub(r'\n+', '\n', text).strip() # Remove extra newlines

return text

? Dynamic Emails List (Will update after each search)

emails = []

? AI Chat Function (Always Uses Latest Email)

def chat_with_ai(question):

"""Sends user question along with the latest fetched email to Groq API and returns AI response."""

if not emails:

return "No emails found. Please search for an email first."

latest_email = emails[-1] # ? Use the latest searched email

email_context = f"Subject: {latest_email['subject']}\n{latest_email['content']}"

prompt = f"""

Here is the latest email:

{email_context}

Answer the user's question based on the email content.

Only return the answer.

Do not include any explanations, thoughts, or <think> sections.

Provide a direct and concise response.


User: {question}

AI:

"""

response = client.chat.completions.create(

model="deepseek-r1-distill-qwen-32b",

messages=[{"role": "user", "content": prompt}],

)

return response.choices[0].message.content.strip()

? Handle User Input

def handle_chat(_):

"""Handles chat interaction when user sends a message."""

user_message = user_input.value.strip()

if not user_message:

return

ai_response = chat_with_ai(user_message) # Get AI response

with chat_history:

print(ai_response) # ? Only print AI response (No <think> parts)

user_input.value = "" # Clear input after sending

? Link Button to Chat Function

send_button.on_click(handle_chat)

? Display UI in Jupyter Notebook

display(chat_history, user_input, send_button)

def enhance_search_query(user_query):

"""AI converts user queries into a correct Gmail search query with simple formatting."""

prompt = f"""

Convert this user request into a valid Gmail API search query.

User query: "{user_query}"

The query should:

Use "subject:", "from:", or general keywords.

Be correctly formatted for Gmail API.

Do NOT use complex logical conditions like "AND" or "OR" inside "subject:".

Do NOT assume an unknown sender (don't add "from:client" unless it's explicitly mentioned).

Keep it as simple as possible.


Example Outputs:

"subject:project update"

"from:dellizo@gmail.com"

"progress OR AI"

"subject:AI progress"


Return ONLY the search query, nothing else.

"""

response = client.chat.completions.create(

model="deepseek-r1-distill-qwen-32b",

messages=[{"role": "user", "content": prompt}],

)

query = response.choices[0].message.content.strip()

? Ensure query does not contain invalid filters

invalid_terms = ["To:", "CC:", "body:", "AND", "OR"]

for term in invalid_terms:

query = query.replace(term, "")

return query

def search_email():

"""Allows user to search manually or use AI. Updates the emails list with the latest fetched

email."""

search_type = input("? Do you want AI to generate a search query? (yes/no): ").strip().lower()

if search_type == "yes":

user_query = input("? Enter your natural language query: ")

optimized_query = enhance_search_query(user_query)

print(f"? AI-Optimized Gmail Query: {optimized_query}")

else:

optimized_query = input("? Enter your own Gmail search query: ").strip()

print(f"? Searching for: {optimized_query}")

Try searching using Gmail API

creds = authenticate_gmail()

service = build("gmail", "v1", credentials=creds)

results = service.users().messages().list(userId="me", q=optimized_query,

maxResults=1).execute()

messages = results.get("messages", [])

if messages:

msg_id = messages[0]["id"]

msg_data = service.users().messages().get(userId="me", id=msg_id).execute()

subject = "No Subject"

for header in msg_data["payload"]["headers"]:

if header["name"] == "Subject":

subject = header["value"]

email_content = extract_body(msg_data["payload"]) # Function must be defined elsewhere

cleaned_content = clean_email_content(email_content)

print(f"? Subject: {subject}\n✉ Content: {cleaned_content[:500]}...\n{'='*50}")

? Update global emails list with the latest fetched email

emails.clear() # Ensure only the latest email is stored

emails.append({"subject": subject, "content": cleaned_content})

return emails[-1]

else:

print("? No emails found in Gmail API. Searching manually...")

If Gmail API fails, fetch recent emails & search manually

user_query = input("? Enter a keyword to search inside fetched emails: ").strip().lower()

fetched_emails = fetch_emails(20) # Fetch last 20 emails (Function must be defined elsewhere)

Search manually inside subjects & body

for email in fetched_emails:

if user_query in email["subject"].lower() or user_query in email["content"].lower():

print(f"? Subject: {email['subject']}\n✉ Content: {email['content'][:5000]}...\n{'='*100}")

? Update global emails list with manually found email

emails.clear()

emails.append({"subject": email["subject"], "content": email["content"]})

return emails[-1]

print("? No matching emails found.")

return None

Example Usage:

searched_email = search_email() # Run once at the start
